In [ ]:
# ============================================================
# NOTEBOOK 08: FRAUD API + INTERACTIVE TESTS
# ============================================================

from flask import Flask, request, jsonify
import numpy as np
from datetime import datetime
import ipywidgets as widgets
from IPython.display import display, clear_output
import threading
import time

# Create Flask app
app = Flask(__name__)

def score_transaction(data):
    """Real-time fraud scoring"""
    amount = float(data.get("amount", 0))
    device = data.get("device_type", "")
    score = 0.1
    if amount > 1000: score += 0.3
    if device in ["unknown", "rooted"]: score += 0.25
    score += np.random.uniform(0, 0.2)
    risk_level = "HIGH" if score > 0.7 else "MEDIUM" if score > 0.4 else "LOW"
    return {
        "risk_score": round(score, 3),
        "risk_level": risk_level,
        "timestamp": datetime.now().isoformat()
    }

@app.route("/", methods=["GET"])
def home():
    return jsonify({"service": "ATO Risk Profiler API", "status": "healthy"})

@app.route("/predict", methods=["POST"])
def predict():
    return jsonify(score_transaction(request.json))

# Interactive widget
print("ATO Risk Profiler API - Interactive Demo")
print("=" * 50)

# Widgets
amount_widget = widgets.FloatSlider(value=250, min=0, max=5000, step=10, description="Amount:")
device_widget = widgets.Dropdown(options=["mobile", "desktop", "unknown", "rooted"], 
                                 value="mobile", description="Device:")
score_button = widgets.Button(description="Score Transaction")

output = widgets.Output()

def on_score_clicked(b):
    with output:
        clear_output()
        tx = {"amount": amount_widget.value, "device_type": device_widget.value}
        result = score_transaction(tx)
        print("TRANSACTION SCORING RESULT:")
        print("-" * 30)
        print(f"Amount: ${tx['amount']:,.0f}")
        print(f"Device: {tx['device_type']}")
        print(f"Risk Score: {result['risk_score']:.1%}")
        print(f"Risk Level: {result['risk_level']}")
        print(f"Time: {result['timestamp'][:19]}")

score_button.on_click(on_score_clicked)

# Display widgets
display(widgets.VBox([amount_widget, device_widget, score_button, output]))

print("\nAPI Endpoints:")
print("- POST /predict")
print("- GET / (status)")